# **Zero shot classification: Demonstrated with the case study "Recipes"**

### All required libraries are imported first

In [ ]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import altair as alt

### The "zero-shot-classification" pipeline of transformers is then defined with a variable

In [ ]:
zer = pipeline("zero-shot-classification")

### Now we define the function that will later process our inputs and deliver the desired result

In [ ]:
def analyze_text(text, labels):
    # If no labels have been selected in the UI, all labels are selected automatically, otherwise the received labels are used
    if not labels:  
        labels = ['Chinese', 'Indian', 'Italian', 
                  'Mexican', 'Southern Usa', 
                  'Cajun Creole', 'French', 'Thai']
        
    # The text input from the UI and the labels are loaded into the zero-shot classification model and the result is then converted into a Pandas DF
    data = zer(text, labels)
    data = pd.DataFrame(data)
    # The score is rounded to two decimal places for the sake of legibility
    data['scores'] = data['scores'].round(2)
    # DF headings are reformatted
    data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                "labels": "Cuisine"})
    # Recipe is removed from the output for the sake of clarity
    data_2 = data.drop(columns=["Recipe"], axis=1)

    # Color palette is created for bar chart
    color_scale = alt.Scale(range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f'])

    # Barplot is created
    plot = alt.Chart(data_2).mark_bar(
        opacity=0.8,
        size= 30
    ).encode(
        x=alt.X('Cuisine', title='Cuisine', sort=alt.EncodingSortField(field='Score',order='descending'), axis=alt.Axis(labelAngle=0, titleAnchor='start')),
        y=alt.Y('Score', title='Score', axis=alt.Axis(titleAnchor='start')),
        color=alt.Color('Cuisine', scale=color_scale),
        tooltip=[alt.Tooltip('Score',title='Matchscore of the Cuisine')]
    ).properties(
    width=600,
    height=400,
    title='Matchscore of the Cuisines'
    ).configure_axis(
    grid=False
    ).configure_view(
    strokeWidth=0
    )

    plot.save('Barplot.html') 

    # The output is returned to the UI
    
    return data_2, plot, 'Barplot.html'

## **Definition of the Gradio UI**

### The checkboxes that can be used to define the labels for the ZSC (if no option is selected, all are selected automatically).

In [ ]:
checkbox = gr.CheckboxGroup(['Chinese','Indian','Italian',
                             'Mexican','Southern Usa', 'Cajun Creole', 
                             'French', 'Thai'], label="Choose countries", info="If nothing is selected, all options will be selected automatically.")

### The Gradio interface is now defined together with the function, inputs, outputs and title and finally executed.

In [ ]:
demo = gr.Interface(
    fn=analyze_text,
    inputs=["text", checkbox],
    outputs=["dataframe","plot","file"],
    title=" Zero-Shot Recipe Classification"
)

demo.launch()

## **____________________________________________________________________________**

## **The entire code in one piece:**

In [ ]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import altair as alt

zer = pipeline("zero-shot-classification")

def analyze_text(text, labels):

    if not labels:  
        labels = ['Chinese', 'Indian', 'Italian', 
                  'Mexican', 'Southern Usa', 
                  'Cajun Creole', 'French', 'Thai']
    data = zer(text, labels)
    data = pd.DataFrame(data)
    data['scores'] = data['scores'].round(2)
    data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                "labels": "Cuisine"})
    data_2 = data.drop(columns=["Recipe"], axis=1)

    color_scale = alt.Scale(range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f'])

    plot = alt.Chart(data_2).mark_bar(
        opacity=0.8,
        size= 30
    ).encode(
        x=alt.X('Cuisine', title='Cuisine', sort=alt.EncodingSortField(field='Score',order='descending'), axis=alt.Axis(labelAngle=0, titleAnchor='start')),
        y=alt.Y('Score', title='Score', axis=alt.Axis(titleAnchor='start')),
        color=alt.Color('Cuisine', scale=color_scale),
        tooltip=[alt.Tooltip('Score',title='Matchscore of the Cuisine')]
    ).properties(
    width=600,
    height=400,
    title='Matchscore of the Cuisines'
    ).configure_axis(
    grid=False
    ).configure_view(
    strokeWidth=0
    )

    plot.save('Barplot.html') 



    return data_2, plot, 'Barplot.html'

checkbox = gr.CheckboxGroup(['Chinese','Indian','Italian',
                             'Mexican','Southern Usa', 'Cajun Creole', 
                             'French', 'Thai'], label="Choose countries", info="If nothing is selected, all options will be selected automatically.")

demo = gr.Interface(
    fn=analyze_text,
    inputs=["text", checkbox],
    outputs=["dataframe","plot","file"],
    title=" Zero-Shot Recipe Classification"
)

demo.launch()